## Case 4

In this tutorial, we provide a demo on how to use FAIRSteam to generate ready-to-model datasets in both TensorFlow Dataset format and DataFrame format. Then, we provide 2 example models that take these modeling data as inputs and make either multi-class or binary classification. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import FAIRStream

In [2]:
# csv pool folder directory
csv_pool_path = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/csv_pool'
# current experiment working directory
work_dir = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_raw'

# initiate a FAIRStream object instance for BSI project
bsi_stream = FAIRStream.FAIRStream(work_dir)
# take a look at dictionaries in engineer's hands
#bsi_stream.engineer.csv_source_dict
print(bsi_stream.engineer.variable_dict)
print(bsi_stream.engineer.csv_source_dict)

Success: Querier has initiated a csv source dictionary in:/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_raw/meta_data/csv_source_dict_demo.json
Success: Querier has initiated a variable dictionary in:/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_raw/meta_data/variable_dict_demo.json
Unable to read sql source dictionary. Use Querier.update_sql_source_dict() to build one.
{'__uid': {'src_names': ['id', 'ID', 'subject_id', 'subject_id'], 'label': 'subject id', 'unique_per_sbj': True}, '__time': {'src_names': ['tsa', 'timeMinutes'], 'label': 'Time since admission', 'unit': 'minute'}, '__anchor': {'src_names': ['True_positive', 'True positive'], 'label': 'episode anchor', 'unique_per_sbj': False, 'factor': {'levels': {'__nbc': ['nan'], '__neg': ['0', '0.0'], '__pos': ['1', '1.0']}, 'impute_per_sbj': {'nan_level': '__nbc'}}}, 'y': {'output': True, 'src_names': ['True_positive', 'True positive'], 'label': 'Blood culture result', 'unique_per_sbj': False, '

In [3]:
# define an episode (notice that the engineer now has new attributes)
bsi_stream.engineer.DefineEpisode(input_time_len=2*24*60, # using vital signs and labs 4 days prior to a culture 
                                  output_time_len=1*24*60, # predict one time unit into the future
                                  time_resolution=60, # aggregate minutely data to one row per hour 
                                  time_lag=0,  # no time lag between predictors and response
                                  anchor_gap=7*24*60) # the minimum distance between two episodes

Success! Engineer has updated attributes --- episode. 


In [ ]:
# Build MVTS dataframe or tfds  (notice that the engineer now has new attributes)
bsi_stream.engineer.BuildMVTS(csv_pool_path, 
                              nsbj = 6000, # number of subjects / patients to sample from the pool 
                              replace=False, # sample with replacement or not 
                              valid_frac = 0.2, # fraction of number of subjects in validation dataset
                              test_frac = 0.1, # fraction of number of subjects in left-out test dataset
                              batch_size = 64, # batch size (usually 32,64,128..)
                              impute_input='median', # imputation on predictors
                              impute_output='median',
                              sep="_")# imputation on response (no need in BSI project)
# please see the end of console 
# --- Success! Engineer has updated attributes --- train_df_imputed, valid_df_imputed and test_df_imputed. 
# --- Success! Engineer has updated attributes --- train_tfds, valid_tfds and test_tfds. 

Engineer is sampling without replacement --- 
6000---out of---11143---subjects are sampled from csv pool of size---31565
-- __time fixed
-- __anchor fixed
-- y fixed
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_7032
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_6881
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_2524
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_3241
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_2811
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_2685
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_82
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_9515
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_2986
--- fix out-of-dictionry level/orders --- [' ']--- with NA for subject ---uva_909
--- fix

--- fix upper boundary for sbp by 234.0
--- fix lower boundary for sbp by 40.0
-- sbp fixed
--- fix upper boundary for sodium by 171.9
--- fix lower boundary for sodium by 115.2
-- sodium fixed
--- fix upper boundary for spo2 by 100.0
--- fix lower boundary for spo2 by 79.00250147058824
-- spo2 fixed
--- fix upper boundary for temp by 104.0
--- fix lower boundary for temp by 90.5
-- temp fixed
--- fix upper boundary for total_bilirubin by 49.6
--- fix lower boundary for total_bilirubin by 0.1
-- total_bilirubin fixed
--- fix upper boundary for total_protein by 10.0
--- fix lower boundary for total_protein by 2.0
-- total_protein fixed
--- fix upper boundary for troponin by 30.0
--- fix lower boundary for troponin by 0.02
-- troponin fixed
--- fix upper boundary for wbc by 50.0
--- fix lower boundary for wbc by 0.05
-- wbc fixed
-- __time fixed
-- __anchor fixed
-- y fixed
--- fix variable dtype for age --- failed.
--- fix upper boundary for age --- failed.
--- fix lower boundary for ag

/Users/jiaxingqiu/Documents/FAIRStream/FAIRStream/Functions/make_episodes_ts.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upper_anchors['__ep_order']=upper_anchors.__time_bin//(anchor_gap//time_resolution)


Success! Output/responce variable mean in current sample space  --- 
y___neg    0.2
y___nbc    0.6
y___pos    0.2
dtype: float64
--- prepare episodes for uvanew_327001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.181818
y___nbc    0.727273
y___pos    0.090909
dtype: float64
--- prepare episodes for uva_2824
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.266667
y___nbc    0.666667
y___pos    0.066667
dtype: float64
--- prepare episodes for uvanew_1300001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.3125
y___nbc    0.6250
y___pos    0.0625
dtype: float64
--- prepare episodes for uva_5067
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.315789
y___nbc    0.631579
y___pos    0.052632
dtype: float64
--- prepare episodes for uvanew_5070001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.35
y___nbc    0.60
y___

Success! Output/responce variable mean in current sample space  --- 
y___neg    0.500000
y___nbc    0.413043
y___pos    0.086957
dtype: float64
--- prepare episodes for uvanew_628001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.500000
y___nbc    0.415493
y___pos    0.084507
dtype: float64
--- prepare episodes for uva_5768
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.503448
y___nbc    0.406897
y___pos    0.089655
dtype: float64
--- prepare episodes for uvanew_475001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.500000
y___nbc    0.414474
y___pos    0.085526
dtype: float64
--- prepare episodes for uvanew_2957001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.509677
y___nbc    0.406452
y___pos    0.083871
dtype: float64
--- prepare episodes for uva_6526
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.512

...

Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523838
y___nbc    0.423054
y___pos    0.053108
dtype: float64
--- prepare episodes for uva_5089
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523666
y___nbc    0.423274
y___pos    0.053060
dtype: float64
--- prepare episodes for uva_1561
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523652
y___nbc    0.423320
y___pos    0.053028
dtype: float64
--- prepare episodes for uvanew_4644001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523795
y___nbc    0.423193
y___pos    0.053012
dtype: float64
--- prepare episodes for uvanew_3892001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523151
y___nbc    0.423933
y___pos    0.052916
dtype: float64
--- prepare episodes for uva_8105
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523294
y__

Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523010
y___nbc    0.423155
y___pos    0.053835
dtype: float64
--- prepare episodes for uva_5740
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523148
y___nbc    0.423032
y___pos    0.053819
dtype: float64
--- prepare episodes for uva_3270
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523273
y___nbc    0.422955
y___pos    0.053773
dtype: float64
--- prepare episodes for uvanew_4442001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523548
y___nbc    0.422710
y___pos    0.053742
dtype: float64
--- prepare episodes for uva_1240
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523219
y___nbc    0.423132
y___pos    0.053649
dtype: float64
--- prepare episodes for uvanew_2927001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.523356
y__

Success! Output/responce variable mean in current sample space  --- 
y___neg    0.526936
y___nbc    0.420875
y___pos    0.052189
dtype: float64
--- prepare episodes for uva_1391
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.527319
y___nbc    0.420566
y___pos    0.052115
dtype: float64
--- prepare episodes for uvanew_1675001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.527451
y___nbc    0.420448
y___pos    0.052101
dtype: float64
--- prepare episodes for uva_6803
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.527583
y___nbc    0.420330
y___pos    0.052086
dtype: float64
--- prepare episodes for uvanew_2337001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.527716
y___nbc    0.420213
y___pos    0.052072
dtype: float64
--- prepare episodes for uva_3735
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.527848
y__

Success! Output/responce variable mean in current sample space  --- 
y___neg    0.526645
y___nbc    0.420881
y___pos    0.052474
dtype: float64
--- prepare episodes for uvanew_4045001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.526902
y___nbc    0.420652
y___pos    0.052446
dtype: float64
--- prepare episodes for uva_1785
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.526730
y___nbc    0.420896
y___pos    0.052374
dtype: float64
--- prepare episodes for uva_351
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.526858
y___nbc    0.420781
y___pos    0.052360
dtype: float64
--- prepare episodes for uva_1395
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.527100
y___nbc    0.420325
y___pos    0.052575
dtype: float64
--- prepare episodes for uva_4568
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.527228
y___nbc    0.

Success! Output/responce variable mean in current sample space  --- 
y___neg    0.528010
y___nbc    0.419372
y___pos    0.052618
dtype: float64
--- prepare episodes for uvanew_2894001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.528134
y___nbc    0.419262
y___pos    0.052604
dtype: float64
--- prepare episodes for uva_6032
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.528119
y___nbc    0.419304
y___pos    0.052577
dtype: float64
--- prepare episodes for uva_442
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.528243
y___nbc    0.419195
y___pos    0.052563
dtype: float64
--- prepare episodes for uvanew_1533001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.528366
y___nbc    0.419085
y___pos    0.052549
dtype: float64
--- prepare episodes for uvanew_314001621
Success! Output/responce variable mean in current sample space  --- 
y___neg    0.528

In [ ]:
all_df = pd.concat( [bsi_stream.engineer.train_df, bsi_stream.engineer.valid_df], axis=0)
all_df = pd.concat( [all_df, bsi_stream.engineer.test_df], axis=0)
print(len(set(all_df.__uid)))
all_df.describe()
all_df.to_csv("./whole_df_raw.csv", index=False)

In [ ]:
# Build MVTS dataframe or tfds  (notice that the engineer now has new attributes)
bsi_stream.engineer.BuildMVTS(csv_pool_path, 
                              nsbj = 6000, # number of subjects / patients to sample from the pool 
                              replace=False, # sample with replacement or not 
                              valid_frac = 0.2, # fraction of number of subjects in validation dataset
                              test_frac = 0.1, # fraction of number of subjects in left-out test dataset
                              batch_size = 64, # batch size (usually 32,64,128..)
                              impute_input='median', # imputation on predictors
                              impute_output='median',
                              sep="_")# imputation on response (no need in BSI project)
# please see the end of console 
# --- Success! Engineer has updated attributes --- train_df_imputed, valid_df_imputed and test_df_imputed. 
# --- Success! Engineer has updated attributes --- train_tfds, valid_tfds and test_tfds. 


In [ ]:
all_df = pd.concat( [all_df, bsi_stream.engineer.train_df], axis=0)
all_df = pd.concat( [all_df, bsi_stream.engineer.valid_df], axis=0)
all_df = pd.concat( [all_df, bsi_stream.engineer.test_df], axis=0)
print(len(set(all_df.__uid)))
all_df.describe()
all_df.to_csv("./whole_df_raw.csv", index=False)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import FAIRStream

## setup

In [2]:
# csv pool folder directory
csv_pool_path = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/csv_pool'
# current experiment working directory
work_dir = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_txp'

In [ ]:
# initiate a FAIRStream object instance for BSI project
bsi_stream = FAIRStream.FAIRStream(work_dir)
# take a look at dictionaries in engineer's hands
#bsi_stream.engineer.csv_source_dict
bsi_stream.engineer.variable_dict

### Add new UVA data to CSV pool

In [4]:
# print( bsi_stream.engineer.csv_source_dict )
# bsi_stream.querier.create_csv_pool(csv_pool_dir = csv_pool_path,
#                                   source_key="uvanew",
#                                   file_key="vital",
#                                   sep="_")

In [5]:
# bsi_stream.querier.create_csv_pool(csv_pool_dir = csv_pool_path,
#                                   source_key="uvanew",
#                                   file_key="lab",
#                                   sep="_")

## Define Episode

In [6]:
# define an episode (notice that the engineer now has new attributes)
bsi_stream.engineer.DefineEpisode(input_time_len=2*24*60, # using vital signs and labs 4 days prior to a culture 
                                  output_time_len=24*60, # predict one time unit into the future
                                  time_resolution=60, # aggregate minutely data to one row per hour 
                                  time_lag=0,  # no time lag between predictors and response
                                  anchor_gap=7*24*60) # the minimum distance between two episodes

Success! Engineer has updated attributes --- episode. 


In [7]:
print(bsi_stream.engineer.episode)

 
An episode is defined to 
--- use 2880 minute(s) long input variables 
--- predict 1440 minute(s) response variables into the future
--- lag 0 minute(s) between predictors and responses
--- increase by every 60 minute(s)
--- last at most 10080 minute(s) long


## Build MVTS (multi-variable time series) data objects
- train_df_imputed, valid_df_imputed and test_df_imputed are dataframes
- train_tfds, valid_tfds and test_tfds are tensorflow datasets

In [ ]:
# Build MVTS dataframe or tfds  (notice that the engineer now has new attributes)
bsi_stream.engineer.BuildMVTS(csv_pool_path, 
                              nsbj = 6000, # number of subjects / patients to sample from the pool 
                              replace=False, # sample with replacement or not 
                              valid_frac = 0.2, # fraction of number of subjects in validation dataset
                              test_frac = 0.1, # fraction of number of subjects in left-out test dataset
                              batch_size = 64, # batch size (usually 32,64,128..)
                              impute_input='median', # imputation on predictors
                              impute_output='median',
                              sep="_")# imputation on response (no need in BSI project)
# please see the end of console 
# --- Success! Engineer has updated attributes --- train_df_imputed, valid_df_imputed and test_df_imputed. 
# --- Success! Engineer has updated attributes --- train_tfds, valid_tfds and test_tfds. 

In [9]:
all_df = pd.concat( [bsi_stream.engineer.train_df, bsi_stream.engineer.valid_df], axis=0)
all_df = pd.concat( [all_df, bsi_stream.engineer.test_df], axis=0)
print(len(set(all_df.__uid)))
all_df.describe()

500


,age,albumin,alp,alt,ast,bicarbonate,bun,calcium,chloride,co2,creatinine,dbp,fio2_pct,glucose,hematocrit,hemoglobin,heart_rate,lactic_acid,magnesium,o2_flow,pco2,peep,ph_arterial,phosphorus,po2,potassium,protime_inr,ptt,resp_rate,sbp,sodium,spo2,temp,total_bilirubin,total_protein,txp___no,txp___yes,troponin,wbc,gram_positive___yes,gram_positive___no,gram_negative___no,gram_negative___yes,fungus___no,fungus___yes,mycobacterial___no,mycobacterial___yes,y___neg,y___nbc,y___pos,__time_bin,__ep_relative_time,__ep_order
count,41616.000000,13845.000000,13848.000000,13573.000000,13426.000000,9326.000000,15501.000000,15501.000000,15501.000000,15491.000000,15501.000000,16870.000000,10574.000000,15884.000000,15977.000000,15953.000000,8594.000000,10891.000000,12813.000000,13493.000000,9372.000000,5796.000000,9372.000000,12991.000000,9323.000000,15798.000000,13736.000000,12898.000000,7808.000000,16870.000000,15885.000000,8255.000000,16625.000000,13844.000000,13700.000000,41616.000000,41616.000000,5692.000000,15498.000000,41616.000000,41616.000000,41616.000000,41616.000000,41616.000000,41616.000000,41616.0,41616.0,41616.000000,41616.000000,41616.000000,41616.000000,41616.000000,41616.000000
mean,57.537900,2.869103,142.406418,73.470431,89.248976,23.561748,32.839244,8.403272,104.557317,22.884702,1.643584,66.180556,35.960376,138.108455,30.967459,10.190063,91.827111,1.955383,1.932647,8.200093,41.164982,7.388529,7.375418,3.597903,117.178569,4.031616,1.487067,35.190082,21.547593,120.928303,137.234589,97.009081,98.368160,2.163170,6.046380,0.956747,0.038062,1.427146,12.161598,0.053633,0.946367,0.922145,0.077855,0.984429,0.015571,1.0,0.0,0.653979,0.202422,0.143599,113.631488,-750.000000,1.228374
std,17.143774,0.681787,131.823721,151.284529,177.867401,5.802400,27.230469,0.874983,6.632194,5.209964,1.631734,13.206883,23.049364,59.058737,6.465586,2.243778,19.582608,1.811709,0.406509,15.298497,10.952069,2.915647,0.092349,1.327419,85.814816,0.608091,0.667794,13.760353,6.711825,23.097684,5.727518,2.592097,1.482321,4.488843,1.174292,0.203428,0.191349,5.537644,7.220936,0.225295,0.225295,0.267946,0.267946,0.123810,0.123810,0.0,0.0,0.475706,0.401810,0.350686,592.087254,1246.971276,0.729817
min,18.811025,1.000000,12.000000,6.000000,7.000000,5.200000,3.000000,4.700000,83.000000,7.000000,0.300000,20.000000,21.000000,43.401900,12.300000,3.600000,37.810271,0.200000,0.900000,0.000000,15.800000,0.000000,6.928000,1.000000,15.100000,2.100000,0.700000,20.200000,6.426743,40.000000,113.000000,79.000000,90.621540,0.100000,2.300000,0.000000,0.000000,0.020000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,-4259.000000,-2880.000000,1.000000
25%,47.234457,2.400000,75.000000,15.000000,22.000000,20.200000,13.000000,7.800000,100.000000,20.000000,0.700000,57.000000,21.000000,101.000000,26.200000,8.600000,77.461548,1.100000,1.650000,0.000000,34.300000,5.000000,7.317000,2.700000,71.400000,3.600000,1.100000,26.200000,16.776906,104.000000,134.000000,95.652110,97.633333,0.500000,5.200000,1.000000,0.000000,0.030000,7.160000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,-25.000000,-1815.000000,1.000000
50%,58.478133,2.800000,102.000000,27.000000,35.000000,23.700000,24.000000,8.400000,104.000000,23.000000,0.975000,65.000000,21.000000,122.000000,30.000000,9.800000,89.647971,1.500000,1.900000,2.000000,40.600000,8.000000,7.389000,3.400000,90.000000,3.990000,1.300000,30.200000,20.543240,118.000000,137.000000,97.507562,98.200000,0.800000,6.000000,1.000000,0.000000,0.060000,10.620000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.0,1.000000,0.000000,0.000000,-4.000000,-750.000000,1.000000
75%,70.014245,3.300000,153.000000,59.000000,78.000000,27.100000,43.000000,9.000000,109.000000,26.000000,1.900000,74.000000,40.000000,156.000000,34.500000,11.400000,105.021942,2.100000,2.200000,6.000000,45.500000,10.000000,7.440000,4.200000,124.100000,4.369167,1.500000,39.100000,25.350873,136.000000,140.00

In [26]:
all_df.to_csv("./whole_df_uva.csv", index=False)

### print all the attributes of the engineer

In [ ]:
print(bsi_stream.engineer)

In [ ]:
bsi_stream.engineer.hist

In [ ]:
# # save raw dataframe in a csv
# bsi_stream.engineer.train_df.to_csv("./train_df_uvanew.csv", index=False)
# bsi_stream.engineer.valid_df.to_csv("./valid_df_uvanew.csv", index=False)
# bsi_stream.engineer.test_df.to_csv("./test_df_uvanew.csv", index=False)
# bsi_stream.engineer.train_df_imputed.to_csv("./train_df_median_uvanew.csv", index=False)
# bsi_stream.engineer.valid_df_imputed.to_csv("./valid_df_median_uvanew.csv", index=False)
# bsi_stream.engineer.test_df_imputed.to_csv("./test_df_median_uvanew.csv", index=False)


In [ ]:
# save tfds in np.array objects
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = bsi_stream.engineer.ExtractXY()
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)
print("X_valid shape", X_valid.shape)
print("Y_valid shape", Y_valid.shape)
print("X_test shape", X_test.shape)
print("Y_test shape", Y_test.shape)
np.save("./X_train_uvanew.npy", X_train)
np.save("./Y_train_uvanew.npy", Y_train)
np.save("./X_valid_uvanew.npy", X_valid)
np.save("./Y_valid_uvanew.npy", Y_valid)
np.save("./X_test_uvanew.npy", X_test)
np.save("./Y_test_uvanew.npy", Y_test)

## Multilabel classification -- by TFDS

In [ ]:
train_tfds = bsi_stream.engineer.train_tfds
valid_tfds = bsi_stream.engineer.valid_tfds
input_shape = list(train_tfds.element_spec[0].shape)[1:3]
myMetrics = [
    keras.metrics.AUC(name='AUROC', curve='ROC'),
    keras.metrics.AUC(name='AUPRC', curve='PR')
]

keras.backend.clear_session()
mdl = keras.models.Sequential([
    keras.layers.Flatten(input_shape=input_shape),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(3, activation='softmax'),
    keras.layers.Reshape([1, -1])
])
mdl.summary()
mdl.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics = myMetrics)
callback = EarlyStopping(monitor='val_loss', patience=10)
his = mdl.fit(train_tfds, 
              epochs=50, 
              validation_data=valid_tfds, callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-4)
his = mdl.fit(train_tfds, 
              epochs=50, 
              validation_data=valid_tfds, callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-5)
his = mdl.fit(train_tfds, 
              epochs=50, 
              validation_data=valid_tfds, callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-6)
his = mdl.fit(train_tfds, 
              epochs=50, 
              validation_data=valid_tfds, callbacks=[callback])

## Binary classification -- by DataFrame

In [ ]:
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = bsi_stream.engineer.ExtractXY()

X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')

Y_train = Y_train.reshape(Y_train.shape[0], Y_train.shape[2])
Y_valid = Y_valid.reshape(Y_valid.shape[0], Y_valid.shape[2])
Y_test = Y_test.reshape(Y_test.shape[0], Y_test.shape[2])
Y_train = Y_train[:,1]
Y_valid = Y_valid[:,1]
Y_test = Y_test[:,1]
Y_train = Y_train.reshape(Y_train.shape[0], 1)
Y_valid = Y_valid.reshape(Y_valid.shape[0], 1)
Y_test = Y_test.reshape(Y_test.shape[0], 1)

X_all = np.concatenate((X_train, X_valid, X_test), axis=0)
Y_all = np.concatenate((Y_train, Y_valid, Y_test), axis=0)

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)
print("X_valid shape", X_valid.shape)
print("Y_valid shape", Y_valid.shape)
print("X_test shape", X_test.shape)
print("Y_test shape", Y_test.shape)
print("X_all shape", X_all.shape)
print("Y_all shape", Y_all.shape)

In [ ]:
myMetrics = [
    keras.metrics.AUC(name='AUROC', curve='ROC'),
    keras.metrics.AUC(name='AUPRC', curve='PR')
]

keras.backend.clear_session()
mdl = keras.models.Sequential([
    keras.layers.LSTM(16, return_sequences=True, input_shape=list(X_train.shape)[1:3]),
    keras.layers.BatchNormalization(),
    keras.layers.LSTM(16, return_sequences=False),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='sigmoid')
])
mdl.summary()
mdl.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics = myMetrics)
callback = EarlyStopping(monitor='val_loss', patience=10)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-4)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-5)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-6)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])